In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 200)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

In [35]:
import pandas as pd
import numpy as np
import glob
import os
import datetime

startDate = '20200916'
endDate = '20200930'



readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
rawOrderLog = []
for thisDate, thisPath in zip(dateLs, dataPathLs):
    data = pd.read_csv(thisPath)
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

# rawOrderLog = pd.read_pickle(r'A:\temp\speedCheck\zt96_statwo_20200818.pkl')


for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

rawOrderLog['clock'] = rawOrderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
rawOrderLog['broker'] = rawOrderLog['accCode'] // 100
rawOrderLog["broker"] = np.where(rawOrderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), rawOrderLog['accCode'] // 10000, rawOrderLog["broker"])
rawOrderLog['colo_broker'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['broker'].astype('str')
rawOrderLog['order'] = rawOrderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
rawOrderLog['group'] = rawOrderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
rawOrderLog['startClock'] = rawOrderLog.groupby(['order'])['clockAtArrival'].transform('first')
rawOrderLog['duration'] = rawOrderLog['clockAtArrival'] - rawOrderLog['startClock']
rawOrderLog['orderPrice'] = rawOrderLog['orderPrice'].apply(lambda x: round(x, 2))
rawOrderLog['tradePrice'] = rawOrderLog['tradePrice'].apply(lambda x: round(x, 2))
rawOrderLog['orderDirection1'] = np.where(rawOrderLog["orderDirection"] == -2, -1, np.where(
    rawOrderLog["orderDirection"] == 2, 1, rawOrderLog["orderDirection"]))
orderLog = rawOrderLog.copy()

### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)][['date', 'accCode', 'secid', 'vai', 'orderDirection', 'order']])
    orderLog = orderLog[orderLog['directNum'] == 1]

assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby('accCode')['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby('accCode')['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby('accCode')['order'].nunique().reset_index()
k2 = orderLog.groupby('accCode')['order'].nunique().reset_index()
k = pd.merge(k1, k2, on='accCode', how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure CYB stocks total absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks total order size < 30w')
try:
    assert(orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)]['absOrderSize'].max() <= 300000)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog['tradeNtl'] = orderLog.groupby('order')['tradeNtl'].transform('sum')
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog['sta'] = np.where(orderLog['ars'].isin([121, 221, 321, 131, 231, 331]), 'statwo', 'staone')
orderLog['cumFillSize'] = orderLog.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
orderLog['status'] = np.where(orderLog['cumFillSize'] == 0, '3. no filled', 
                                np.where(orderLog['cumFillSize'] < orderLog['absOrderSize'], '2. partial filled', '1. full filled'))


startDate = '20200916'
endDate = '20200930'
readPath = r'L:\orderLog\result\marketPos'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'SZspeed_***.pkl')))
dateLs = np.array([os.path.basename(i).split('.')[0].split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
checkData = []
for path in dataPathLs:
    data = pd.read_pickle(path)
    checkData += [data]
checkLog = pd.concat(checkData).reset_index(drop=True)   
checkLog['date'].unique()

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,19,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1. same date, secid, vai: same direction
opposite direction for same date, same secid, same vai


,date,accCode,secid,vai,orderDirection,order
25078,20200916,9248,1600650,1273426,1,38363
25082,20200916,9741,1600650,1273426,-1,44617
25086,20200916,522501,1600650,1273426,-1,54727
25089,20200916,527601,1600650,1273426,-1,58474
25095,20200916,529001,1600650,1273426,-1,63585
25098,20200916,975602,1600650,1273426,-1,67738
71034,20200916,8854,1603606,11802970,-1,32502
71043,20200916,9551,1603606,11802970,1,42827
214422,20200916,8854,2300326,18003082,1,32930
214427,20200916,9685,2300326,18003082,-1,44071


2. same date, secid, vai, accCode: one insertion
more than one insertion at same time


,date,accCode,secid,vai,order,clockAtArrival
0,20200916,5225,2002805,98400,2173,2
1,20200916,6683,2300725,1374239,31106,2
2,20200916,6683,2300788,333400,31192,2
3,20200916,8854,1603429,5619973,32484,2
4,20200916,8854,2300788,445700,33084,2
...,...,...,...,...,...,...
488,20200930,9741,2002735,1029020,762557,2
489,20200930,9741,2002802,340600,762660,2
490,20200930,9741,2300492,151700,763460,2
491,20200930,9741,2300522,1074820,763546,2


...

,order,filled,total


10. no cancellation within 1 sec
any cancellation within 1 sec


,Unnamed: 0,clockAtArrival,caamd,secid,updateType,vai,ars,absFilledThisUpdate,orderDirection,absOrderSize,absOrderSizeCumFilled,orderPrice,tradePrice,date,accCode,mse,colo,orderSysId,internalId,tradeId,sdd,aaa,ApplSeqNum,mrm,mta,mrsb,mrss,mrv,mrb100,mra100,l4tr,clock,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status


11. Orders with size > 80w or notional > 800w


array([20200916, 20200917, 20200918, 20200921, 20200922, 20200923,
       20200924, 20200925, 20200928, 20200929, 20200930], dtype=int64)

In [36]:
checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog = checkLog[~checkLog['accCode'].isnull()]
targetStockSZ = checkLog['SecurityID'].unique()

In [42]:
startTm = datetime.datetime.now()

startDate = '20200916'
endDate = '20200930'

readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]
OrderLog = []
TradeLog = []
for date in dateLs:
    readPath = 'A:\\rawData'
    mdOrderLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdOrderLog***.csv'))[-1]
    OrderLog1 = pd.read_csv(mdOrderLogPath)
    OrderLog1 = OrderLog1[OrderLog1['SecurityID'].isin(targetStockSZ)]
    OrderLog1['OrderType'] = OrderLog1['OrderType'].astype(str)
    OrderLog1 = OrderLog1[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'Side',
                             'OrderType', 'Price', 'OrderQty']]
    OrderLog1['date'] = int(date)
    OrderLog += [OrderLog1]
    del OrderLog1

    
    mdTradeLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdTradeLog***.csv'))[-1]
    TradeLog1 = pd.read_csv(mdTradeLogPath, encoding='utf-8')
    TradeLog1['ExecType'] = TradeLog1['ExecType'].astype(str)
    TradeLog1 = TradeLog1[TradeLog1['SecurityID'].isin(targetStockSZ)]
    TradeLog1 = TradeLog1[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum',
                             'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 'OfferApplSeqNum']]
    TradeLog1['date'] = int(date)
    TradeLog += [TradeLog1]
OrderLog = pd.concat(OrderLog).reset_index(drop=True)
TradeLog = pd.concat(TradeLog).reset_index(drop=True)
checkLog = checkLog[['date', 'SecurityID', 'sequenceNo', 'colo', 'accCode', 'secid', 'ApplSeqNum', 'Side']]
checkLog = pd.merge(checkLog, OrderLog[['SecurityID', 'date', 'sequenceNo', 'ApplSeqNum']],
             on=['date', 'SecurityID', 'sequenceNo'], validate='one_to_one')
checkLog = checkLog.rename(columns={'ApplSeqNum_y':'ApplSeqNum_order', 'ApplSeqNum_x':'ApplSeqNum'})
checkLog = pd.merge(checkLog, OrderLog[['SecurityID', 'date', 'ApplSeqNum', 'clockAtArrival']],
             on=['date', 'SecurityID', 'ApplSeqNum'], validate='many_to_one')
checkLog = checkLog.rename(columns={'clockAtArrival':'market_insert'})

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MemoryError: 

In [5]:
orderLog['ApplSeqNum'] = orderLog.groupby(['order'])['ApplSeqNum'].transform('first')
orderLog['caamd'] = orderLog.groupby(['order'])['caamd'].transform('first')
checkLog = pd.merge(checkLog, orderLog[(orderLog['exchange'] == 'SZE')
                                & (orderLog['ApplSeqNum'] != 0)][['date', 'colo', 'accCode', 'secid', 'ApplSeqNum', 'clockAtArrival', 'caamd', 'updateType', 'order']],
             on=['date', 'colo', 'accCode', 'secid', 'ApplSeqNum'], validate='one_to_many')
checkLog = checkLog.rename(columns={'clockAtArrival':'internal_trade', 'caamd':'internal_insert'})
checkLog = checkLog[checkLog['updateType'] == 4]
checkLog1 = checkLog[checkLog['Side'] == 1]
checkLog2 = checkLog[checkLog['Side'] == 2]

In [6]:
checkLog1 = pd.merge(checkLog1, TradeLog,
             left_on=['date', 'SecurityID', 'ApplSeqNum_order'], right_on=['date', 'SecurityID', 'BidApplSeqNum'], validate='many_to_many')
checkLog1 = checkLog1.rename(columns={'clockAtArrival':"market_trade", 'sequenceNo_x':'sequenceNo_order','sequenceNo_y':'sequenceNo_trade'})
checkLog1 = checkLog1.sort_values(by=['date', 'secid', 'ApplSeqNum_order', 'internal_trade'])
checkLog1['count1'] = checkLog1.groupby(['date', 'secid', 'ApplSeqNum_order', 'internal_trade']).cumcount()
checkLog1 = checkLog1.sort_values(by=['date', 'secid', 'ApplSeqNum_order', 'sequenceNo_trade'])
checkLog1['count2'] = checkLog1.groupby(['date', 'secid', 'ApplSeqNum_order', 'sequenceNo_trade']).cumcount()
c1 = checkLog1.groupby(['date', 'secid', 'ApplSeqNum_order'])['count1'].max().reset_index()
c2 = checkLog1.groupby(['date', 'secid', 'ApplSeqNum_order'])['count2'].max().reset_index()
cc = pd.merge(c1, c2, on=['date', 'secid', 'ApplSeqNum_order'])
assert(cc[cc['count1'] != cc['count2']].shape[0] == 0)
checkLog1 = checkLog1[checkLog1['count1'] == checkLog1['count2']]

In [7]:
checkLog2 = pd.merge(checkLog2, TradeLog[['SecurityID', 'date', 'clockAtArrival', 'OfferApplSeqNum', 'sequenceNo']],
             left_on=['date', 'SecurityID', 'ApplSeqNum_order'], right_on=['date', 'SecurityID', 'OfferApplSeqNum'], validate='many_to_many')
del TradeLog
checkLog2 = checkLog2.rename(columns={'clockAtArrival':"market_trade", 'sequenceNo_x':'sequenceNo_order','sequenceNo_y':'sequenceNo_trade'})
checkLog2 = checkLog2.sort_values(by=['date', 'secid', 'ApplSeqNum_order', 'internal_trade'])
checkLog2['count1'] = checkLog2.groupby(['date', 'secid', 'ApplSeqNum_order', 'internal_trade']).cumcount()
checkLog2 = checkLog2.sort_values(by=['date', 'secid', 'ApplSeqNum_order', 'sequenceNo_trade'])
checkLog2['count2'] = checkLog2.groupby(['date', 'secid', 'ApplSeqNum_order', 'sequenceNo_trade']).cumcount()
c1 = checkLog2.groupby(['date', 'secid', 'ApplSeqNum_order'])['count1'].max().reset_index()
c2 = checkLog2.groupby(['date', 'secid', 'ApplSeqNum_order'])['count2'].max().reset_index()
cc = pd.merge(c1, c2, on=['date', 'secid', 'ApplSeqNum_order'])
assert(cc[cc['count1'] != cc['count2']].shape[0] == 0)
checkLog2 = checkLog2[checkLog2['count1'] == checkLog2['count2']]

In [8]:
checkLog = pd.concat([checkLog1, checkLog2])
checkLog['interval1'] = checkLog['internal_trade'] - checkLog['internal_insert']
checkLog['interval2'] = checkLog['market_trade'] - checkLog['market_insert']
checkLog['diff'] = checkLog['interval1'] - checkLog['interval2']

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
checkLog.head(50)

,ApplSeqNum,ApplSeqNum_order,BidApplSeqNum,OfferApplSeqNum,SecurityID,Side,accCode,colo,count1,count2,date,internal_insert,internal_trade,market_insert,market_trade,order,secid,sequenceNo_order,sequenceNo_trade,updateType,interval1,interval2,diff
0,8468262.0,8468265,8468265.0,NaN,1,1.0,9655.0,zs_96_02,0,0,20200930,1601434639762749,1601434639765727,1601434642603108,1601434642605758,41896,2000001,50674895,50674896,4,2978,2650,328
1,2008810.0,2008817,2008817.0,NaN,11,1.0,5275.0,zs_52_06,0,0,20200930,1601429827766694,1601429827769748,1601429830962514,1601429830965427,11324,2000011,10592096,10592097,4,3054,2913,141
2,2013151.0,2013159,2013159.0,NaN,11,1.0,9758.0,zs_96_06,0,0,20200930,1601429829638366,1601429829640947,1601429832699719,1601429832702281,50915,2000011,10614518,10614521,4,2581,2562,19
3,2013151.0,2013161,2013161.0,NaN,11,1.0,5275.0,zs_52_06,0,0,20200930,1601429829503876,1601429829506836,1601429832699719,1601429832702390,11325,2000011,10614522,10614523,4,2960,2671,289
4,8220840.0,8220849,8220849.0,NaN,11,1.0,9208.0,zs_92_01,0,0,20200930,1601434206407455,1601434206410112,1601434206407456,1601434206410256,37587,2000011,48314653,48314654,4,2657,2800,-143
5,8225882.0,8225895,8225895.0,NaN,11,1.0,5275.0,zs_52_06,0,0,20200930,1601434206355440,1601434206358514,1601434209572101,1601434209574895,11331,2000011,48341887,48341888,4,3074,2794,280
8,8225882.0,8225895,8225895.0,NaN,11,1.0,5275.0,zs_52_06,1,1,20200930,1601434206355440,1601434206358572,1601434209572101,1601434209574896,11331,2000011,48341887,48341889,4,3132,2795,337
9,13363259.0,13363284,13363284.0,NaN,11,1.0,9685.0,zs_96_08,0,0,20200930,1601445048940636,1601445048944356,1601445052010758,1601445052014447,42005,2000011,82689283,82689284,4,3720,3689,31
10,13397536.0,13397554,13397554.0,NaN,11,1.0,9685.0,zs_96_08,0,0,20200930,1601445066934446,1601445066937793,1601445070004570,1601445070007912,42006,2000011,82907265,82907266,4,3347,3342,5
11,13445031.0,13445040,13445040.0,NaN,11,1.0,9685.0,zs_96_08,0,0,20200930,1601445087182685,1601445087185465,1601445090252812,1601445090255471,42007,2000011,83169164,83169165,4,2780,2659,121


In [21]:
orderlog = pd.read_csv(r'A:\orderLog\equityTradeLogs\speedCompare_20200930.csv')

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
order = pd.read_csv(r'A:\rawData\logs_20200930_zs_92_01_day_data\mdOrderLog_20200930_0832.csv')

In [23]:
trade = pd.read_csv(r'A:\rawData\logs_20200930_zs_92_01_day_data\mdTradeLog_20200930_0832.csv')

In [26]:
orderlog[(orderlog['accCode'] == 5276) & (orderlog['internalId'] == 532)]


,Unnamed: 0,clockAtArrival,caamd,secid,updateType,vai,ars,absFilledThisUpdate,orderDirection,absOrderSize,absOrderSizeCumFilled,orderPrice,tradePrice,date,accCode,mse,colo,orderSysId,internalId,tradeId,sdd,aaa,ApplSeqNum,mrm,mta,mrsb,mrss,mrv,mrb100,mra100,l4tr
100006,100006,1601429826461803,1601429826461748,2000021,0,2463700,131.0,0,1,8500,0,20.58,-1.00,20200930,5276,100,zs_52_08,NaN,532.0,NaN,93706450.0,0.001593,2099845.0,0.001167,0.001167,0.001593,-0.002027,2463700.0,2057.0,2058.0,0.0
100007,100007,1601429826465506,1601429826465228,2000021,2,2463700,-1.0,0,1,8500,0,20.58,-1.00,20200930,5276,0,zs_52_08,261859.0,532.0,NaN,-1.0,-1.000000,0.0,0.001167,-1.000000,0.001490,-0.002358,2472200.0,2057.0,2059.0,0.0
100008,100008,1601429826465616,1601429826465544,2000021,4,2463700,-1.0,8500,1,8500,8500,20.58,20.58,20200930,5276,0,zs_52_08,NaN,532.0,103000002857721.0,-1.0,-1.000000,0.0,0.001167,-1.000000,0.001561,-0.002024,2472200.0,2058.0,2059.0,0.0


In [28]:
order[(order['ApplSeqNum'] == 2099845) & (order['SecurityID'] == 21)]

,clockAtArrival,sequenceNo,exchId,securityType,__isRepeated,TransactTime,ChannelNo,ApplSeqNum,SecurityID,secid,mdSource,Side,OrderType,__origTickSeq,Price,OrderQty
4717245,1601429829514009,10559127,2,1,0,93706450,2013,2099845,21,2000021,4,1,2,20579657,205700,1300


In [29]:
order[(order['ApplSeqNum'] == 2099863) & (order['SecurityID'] == 21)]

,clockAtArrival,sequenceNo,exchId,securityType,__isRepeated,TransactTime,ChannelNo,ApplSeqNum,SecurityID,secid,mdSource,Side,OrderType,__origTickSeq,Price,OrderQty
4717268,1601429829517403,10559172,2,1,0,93706460,2013,2099863,21,2000021,4,1,2,20579723,205800,8500


In [31]:
trade[(trade['BidApplSeqNum'] == 2099863) & (trade['SecurityID'] == 21)]

,clockAtArrival,sequenceNo,exchId,securityType,__isRepeated,TransactTime,ChannelNo,ApplSeqNum,SecurityID,secid,mdSource,ExecType,TradeBSFlag,__origTickSeq,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum
4092037,1601429829517405,10559173,2,1,0,93706460,2013,2099864,21,2000021,4,F,N,20579724,205800,8500,1749300000,2099863,1665261


In [32]:
1601429829517405 - 1601429829514009

3396

In [33]:
1601429826465616 - 1601429826461748

3868

In [ ]:
checkLog['diff'].describe([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]).astype(np.int64)

In [77]:
checkLog[checkLog['diff'] == -89187]

,ApplSeqNum,ApplSeqNum_order,BidApplSeqNum,OfferApplSeqNum,SecurityID,Side,accCode,colo,count1,count2,date,internal_insert,internal_trade,market_insert,market_trade,order,secid,sequenceNo_order,sequenceNo_trade,updateType,interval1,interval2,diff
75098,10034020.0,10037301,NaN,10037301.0,2791,2.0,6683,zs_66_01,0,0,20200930,1601442002886844,1601442002890297,1601442005197716,1601442005290356,29645,2002791,61580315,61580316,4,3453,92640,-89187


In [16]:
orderLog[(orderLog['secid'] == 2000011) & (orderLog['ApplSeqNum'] == 8225882) & (orderLog['accCode'] == 5275)]

,Unnamed: 0,clockAtArrival,caamd,secid,updateType,vai,ars,absFilledThisUpdate,orderDirection,absOrderSize,absOrderSizeCumFilled,orderPrice,tradePrice,date,accCode,mse,colo,orderSysId,internalId,tradeId,sdd,aaa,ApplSeqNum,mrm,mta,mrsb,mrss,mrv,mrb100,mra100,l4tr,clock,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status,orderNtl,exchange,tradeNtl,sta,cumFillSize
99375,99694,1601434206355483,1601434206355440,2000011,0,5809000,131.0,0,1,2200,0,17.69,-1.00,20200930,5275,100,zs_52_06,NaN,3783.0,NaN,105006500.0,0.001578,8225882.0,0.001742,0.001742,0.001578,-0.002188,5809000.0,1768.0,1769.0,0.0,2020-09-30 10:50:06.355483,52,zs_52,11331,12512,1601434206355483,0,1,1,1.0,1. full filled,38918.0,SZE,38918.0,statwo,2200
99376,99695,1601434206358289,1601434206355440,2000011,2,5809000,131.0,0,1,2200,0,17.69,-1.00,20200930,5275,0,zs_52_06,836718.0,3783.0,NaN,-1.0,-1.000000,8225882.0,0.001742,-1.000000,-1.000000,-1.000000,5811600.0,1768.0,1769.0,0.0,2020-09-30 10:50:06.358289,52,zs_52,11331,12512,1601434206355483,2806,1,1,1.0,1. full filled,38918.0,SZE,38918.0,statwo,2200
99377,99696,1601434206358514,1601434206355440,2000011,4,5809000,131.0,1500,1,2200,1500,17.69,17.69,20200930,5275,0,zs_52_06,836718.0,3783.0,104000011633104.0,-1.0,-1.000000,8225882.0,0.001742,-1.000000,-1.000000,-1.000000,5811600.0,1768.0,1769.0,0.0,2020-09-30 10:50:06.358514,52,zs_52,11331,12512,1601434206355483,3031,1,1,1.0,1. full filled,38918.0,SZE,38918.0,statwo,2200
99378,99697,1601434206358572,1601434206355440,2000011,4,5809000,131.0,700,1,2200,2200,17.69,17.69,20200930,5275,0,zs_52_06,NaN,3783.0,104000011633106.0,-1.0,-1.000000,8225882.0,0.001742,-1.000000,-1.000000,-1.000000,5811600.0,1768.0,1769.0,0.0,2020-09-30 10:50:06.358572,52,zs_52,11331,12512,1601434206355483,3089,1,1,1.0,1. full filled,38918.0,SZE,38918.0,statwo,2200


In [20]:
1601434206358514 - 1601434206355440

3074

In [19]:
1601434209574895 - 1601434209572101

2794

In [18]:
TradeLog[(TradeLog['BidApplSeqNum'] == 8225895) & (TradeLog['SecurityID'] == 11)]

,clockAtArrival,sequenceNo,TransactTime,SecurityID,ApplSeqNum,ExecType,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,date
11139888,1601434209574895,48341888,105006510,11,8225896,F,176900,1500,265350000,8225895,8176586,20200930
11139889,1601434209574896,48341889,105006510,11,8225897,F,176900,700,123830000,8225895,8176824,20200930


In [13]:
OrderLog[(OrderLog['ApplSeqNum'] == 8225882) & (OrderLog['SecurityID'] == 11)]

,clockAtArrival,sequenceNo,TransactTime,SecurityID,ApplSeqNum,Side,OrderType,Price,OrderQty,date
13465433,1601434209572101,48341855,105006500,11,8225882,1,2,176900,1700,20200930


In [80]:
from IPython.display import display, HTML
checkLog['accCode'] = checkLog['accCode'].astype(int)
HTML(checkLog.groupby(['colo'])['diff'].describe().sort_values(by='mean').astype(int).to_html())

,count,mean,std,min,25%,50%,75%,max
colo,,,,,,,,
zt_96_01,100,-1630,1285,-7117,-2336,-1576,-1109,2524
zs_64_01,420,-642,629,-6777,-823,-509,-309,672
zs_94_03,248,-344,988,-4299,-338,-170,-47,9605
zs_92_02,198,-223,195,-2104,-316,-224,-106,200
zs_92_01,55,-149,141,-477,-246,-153,-49,198
zs_66_01,3402,-73,1939,-89187,-132,-26,93,5312
zs_94_05,1906,-7,238,-2407,-127,-19,97,2965
zs_96_06,2661,68,728,-30867,-40,67,168,12293
zs_96_02,99,69,114,-215,16,78,131,338
